<a href="https://colab.research.google.com/github/Quanghoaai/Podo/blob/main/YouTube_to_chatbot_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Updated Code for Article Processing with Secure API Key Input

In [ ]:
# Install dependencies
!pip install -qU \
    pinecone-client[grpc]==2.2.1 \
    langchain==0.0.162 \
    tiktoken==0.4.0 \
    datasets==2.12.0 \
    youtube_transcript_api \
    google-api-python-client \
    google-generativeai

# Import libraries
import os
import google.generativeai as genai
import tiktoken
from pathlib import Path
from tqdm import tqdm
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

# Extract video ID from a YouTube URL
def extract_video_id_from_url(video_url):
    """
    Extracts the video ID from a YouTube video URL.
    Example: https://www.youtube.com/watch?v=dQw4w9WgXcQ -> 'dQw4w9WgXcQ'
    """
    parsed_url = urlparse(video_url)
    if parsed_url.hostname == 'youtu.be':  # Shortened URL format
        return parsed_url.path[1:]  # Remove the leading '/'
    elif parsed_url.hostname in ('www.youtube.com', 'youtube.com'):
        if parsed_url.path == '/watch':
            return parse_qs(parsed_url.query)['v'][0]
        elif parsed_url.path.startswith('/embed/'):  # Embedded video URL
            return parsed_url.path.split('/')[2]
    return None

# Get transcript for a specific video in multiple languages
def get_transcript_for_video(video_id, languages=['vi', 'en']):
    transcripts = {}
    for lang in languages:
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])
            transcripts[lang] = transcript
            print(f"Successfully fetched transcript for language: {lang}")
        except Exception as ex:
            print(f"No transcript available for language: {lang} [{ex}]")
            transcripts[lang] = []
    return transcripts

# Combine transcripts from multiple languages into one text
def combine_transcripts(transcripts):
    combined_text = ""
    for lang, transcript in transcripts.items():
        combined_text += f"\n--- Language: {lang.upper()} ---\n"
        for item in transcript:
            combined_text += item['text'] + '\n'
    return combined_text

# Write transcript to file
def write_to_file(text, output_file="YouTube.txt"):
    with open(output_file, 'w') as f:
        f.write(text)

# Load text from file
def load_text(file_path):
    with Path(file_path).open("r") as file:
        return file.read()

# Save responses to file
def save_to_file(responses, output_file):
    with Path(output_file).open('w') as file:
        file.write("\n".join(responses))

# Call Gemini API to clean text
def call_gemini_api(chunk, model):
    """Calls the Gemini API to clean the given chunk of text."""
    prompt = f"""Clean the following transcripts of all grammatical mistakes, misplaced words, and identify the speakers:
    {chunk}
    """
    try:
        response = model.generate_content(prompt)
        response.resolve()  # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

# Split text into chunks
def split_into_chunks(text, n_tokens=300):
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')  # Using gpt-3.5-turbo tokenizer for token estimation
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), n_tokens):
        chunk_tokens = tokens[i:i + n_tokens]
        try:
            chunks.append(encoding.decode(chunk_tokens))
        except Exception as e:
            print(f"Error decoding tokens: {e}")
            continue
    return chunks

# Process chunks through Gemini API
def process_chunks(input_file, output_file, model, delay=0):  # delay in seconds (if you hit a rate limit error)
    text = load_text(input_file)
    chunks = split_into_chunks(text)[:5]
    responses = []
    for chunk in tqdm(chunks):
        responses.append(call_gemini_api(chunk, model))
    save_to_file(responses, output_file)

# Clean user query
def clean_query(query, model):
    prompt = f"""Clean the following query to make sure it is grammatically correct and can retrieve the correct information: {query}"""
    try:
        response = model.generate_content(prompt)
        response.resolve()  # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

# Query knowledge base
def query_knowledge_base(query, knowledge_base, model):
    """
    Queries the knowledge base using the Gemini API.
    """
    context = "\n".join(knowledge_base)
    prompt = f"""You are a chatbot that answers questions to the best of your ability using the context provided.
    Context:
    {context}
    Question: {query}
    """
    try:
        response = model.generate_content(prompt)
        response.resolve()  # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

# Conversational agent
def conversational_agent(query, knowledge_base, chat_history, model):
    """
    A conversational agent that uses the Gemini API to answer questions.
    """
    # Clean query before processing
    cleaned_query = clean_query(query, model)
    # Query the knowledge base
    response = query_knowledge_base(cleaned_query, knowledge_base, model)
    # Update the chat history
    chat_history.append(f"User: {query}")
    chat_history.append(f"Assistant: {response}")
    return response, chat_history

# Main function
if __name__ == "__main__":
    # Step 1: Request API key from the user
    GOOGLE_API_KEY = input("Please enter your Gemini API key: ").strip()
    if not GOOGLE_API_KEY:
        print("API key is required to proceed.")
        exit()

    # Configure Gemini API with the provided key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Or another suitable Gemini model

    # Step 2: Provide a YouTube video URL
    video_url = input("Please enter the YouTube video URL: ").strip()
    video_id = extract_video_id_from_url(video_url)

    if not video_id:
        print("Invalid YouTube URL")
    else:
        print(f"Extracted Video ID: {video_id}")

        # Step 3: Get transcript in Vietnamese and English
        transcripts = get_transcript_for_video(video_id, languages=['vi', 'en'])

        # Step 4: Combine transcripts into one text
        combined_text = combine_transcripts(transcripts)

        # Step 5: Write combined transcript to file
        write_to_file(combined_text, "YouTube.txt")
        print("Combined transcript has been written to YouTube.txt")

        # Step 6: Process transcript through Gemini API
        input_file = "YouTube.txt"
        output_file = "clean_transcript.txt"
        process_chunks(input_file, output_file, model)

        # Step 7: Build knowledge base
        with Path('clean_transcript.txt').open('r') as file:
            lines = file.read().splitlines()
        chunks = [' '.join(lines[i:i+5]) for i in range(0, len(lines), 5)]

        # Step 8: Use conversational agent
        knowledge_base = chunks.copy()
        chat_history = []

        while True:
            query = input("Ask a question (or type 'exit' to quit): ").strip()
            if query.lower() == 'exit':
                break

            response, chat_history = conversational_agent(query, knowledge_base, chat_history, model)
            print(response)

        print(chat_history)

1.Full Code for Multi-Language Transcript (Vietnamese and English)
python


In [ ]:
# Install dependencies
!pip install -qU \
    pinecone-client[grpc]==2.2.1 \
    langchain==0.0.162 \
    tiktoken==0.4.0 \
    datasets==2.12.0 \
    youtube_transcript_api \
    google-api-python-client \
    google-generativeai

# Import libraries
import os
import google.generativeai as genai
import tiktoken
from pathlib import Path
from tqdm import tqdm
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

# Extract video ID from a YouTube URL
def extract_video_id_from_url(video_url):
    """
    Extracts the video ID from a YouTube video URL.
    Example: https://www.youtube.com/watch?v=dQw4w9WgXcQ -> 'dQw4w9WgXcQ'
    """
    parsed_url = urlparse(video_url)
    if parsed_url.hostname == 'youtu.be':  # Shortened URL format
        return parsed_url.path[1:]  # Remove the leading '/'
    elif parsed_url.hostname in ('www.youtube.com', 'youtube.com'):
        if parsed_url.path == '/watch':
            return parse_qs(parsed_url.query)['v'][0]
        elif parsed_url.path.startswith('/embed/'):  # Embedded video URL
            return parsed_url.path.split('/')[2]
    return None

# Get transcript for a specific video in multiple languages
def get_transcript_for_video(video_id, languages=['vi', 'en']):
    transcripts = {}
    for lang in languages:
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[lang])
            transcripts[lang] = transcript
            print(f"Successfully fetched transcript for language: {lang}")
        except Exception as ex:
            print(f"No transcript available for language: {lang} [{ex}]")
            transcripts[lang] = []
    return transcripts

# Combine transcripts from multiple languages into one text
def combine_transcripts(transcripts):
    combined_text = ""
    for lang, transcript in transcripts.items():
        combined_text += f"\n--- Language: {lang.upper()} ---\n"
        for item in transcript:
            combined_text += item['text'] + '\n'
    return combined_text

# Write transcript to file
def write_to_file(text, output_file="YouTube.txt"):
    with open(output_file, 'w') as f:
        f.write(text)

# Load text from file
def load_text(file_path):
    with Path(file_path).open("r") as file:
        return file.read()

# Save responses to file
def save_to_file(responses, output_file):
    with Path(output_file).open('w') as file:
        file.write("\n".join(responses))

# Call Gemini API to clean text
def call_gemini_api(chunk, model):
    """Calls the Gemini API to clean the given chunk of text."""
    prompt = f"""Clean the following transcripts of all grammatical mistakes, misplaced words, and identify the speakers:
    {chunk}
    """
    try:
        response = model.generate_content(prompt)
        response.resolve()  # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

# Split text into chunks
def split_into_chunks(text, n_tokens=300):
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')  # Using gpt-3.5-turbo tokenizer for token estimation
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), n_tokens):
        chunk_tokens = tokens[i:i + n_tokens]
        try:
            chunks.append(encoding.decode(chunk_tokens))
        except Exception as e:
            print(f"Error decoding tokens: {e}")
            continue
    return chunks

# Process chunks through Gemini API
def process_chunks(input_file, output_file, model, delay=0):  # delay in seconds (if you hit a rate limit error)
    text = load_text(input_file)
    chunks = split_into_chunks(text)[:5]
    responses = []
    for chunk in tqdm(chunks):
        responses.append(call_gemini_api(chunk, model))
    save_to_file(responses, output_file)

# Clean user query
def clean_query(query, model):
    prompt = f"""Clean the following query to make sure it is grammatically correct and can retrieve the correct information: {query}"""
    try:
        response = model.generate_content(prompt)
        response.resolve()  # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

# Query knowledge base
def query_knowledge_base(query, knowledge_base, model):
    """
    Queries the knowledge base using the Gemini API.
    """
    context = "\n".join(knowledge_base)
    prompt = f"""You are a chatbot that answers questions to the best of your ability using the context provided.
    Context:
    {context}
    Question: {query}
    """
    try:
        response = model.generate_content(prompt)
        response.resolve()  # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

# Conversational agent
def conversational_agent(query, knowledge_base, chat_history, model):
    """
    A conversational agent that uses the Gemini API to answer questions.
    """
    # Clean query before processing
    cleaned_query = clean_query(query, model)
    # Query the knowledge base
    response = query_knowledge_base(cleaned_query, knowledge_base, model)
    # Update the chat history
    chat_history.append(f"User: {query}")
    chat_history.append(f"Assistant: {response}")
    return response, chat_history

# Main function
if __name__ == "__main__":
    # Step 1: Request API key from the user
    GOOGLE_API_KEY = input("Please enter your Gemini API key: ").strip()
    if not GOOGLE_API_KEY:
        print("API key is required to proceed.")
        exit()

    # Configure Gemini API with the provided key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Or another suitable Gemini model

    # Step 2: Provide a YouTube video URL
    video_url = input("Please enter the YouTube video URL: ").strip()
    video_id = extract_video_id_from_url(video_url)

    if not video_id:
        print("Invalid YouTube URL")
    else:
        print(f"Extracted Video ID: {video_id}")

        # Step 3: Get transcript in Vietnamese and English
        transcripts = get_transcript_for_video(video_id, languages=['vi', 'en'])

        # Step 4: Combine transcripts into one text
        combined_text = combine_transcripts(transcripts)

        # Step 5: Write combined transcript to file
        write_to_file(combined_text, "YouTube.txt")
        print("Combined transcript has been written to YouTube.txt")

        # Step 6: Process transcript through Gemini API
        input_file = "YouTube.txt"
        output_file = "clean_transcript.txt"
        process_chunks(input_file, output_file, model)

        # Step 7: Build knowledge base
        with Path('clean_transcript.txt').open('r') as file:
            lines = file.read().splitlines()
        chunks = [' '.join(lines[i:i+5]) for i in range(0, len(lines), 5)]

        # Step 8: Use conversational agent
        knowledge_base = chunks.copy()
        chat_history = []

        while True:
            query = input("Ask a question (or type 'exit' to quit): ").strip()
            if query.lower() == 'exit':
                break

            response, chat_history = conversational_agent(query, knowledge_base, chat_history, model)
            print(response)

        print(chat_history)

## Install dependencies

In [ ]:
!pip install -qU \
    openai==0.27.7 \
    pinecone-client[grpc]==2.2.1 \
    langchain==0.0.162 \
    tiktoken==0.4.0 \
    datasets==2.12.0 \
    youtube_transcript_api

# Scrape an entire YouTube channel into one .txt file

In [ ]:
!pip install google-api-python-client
import googleapiclient.discovery
from tqdm import tqdm
from youtube_transcript_api import YouTubeTranscriptApi
api_key = "" #@param {type:"string"}
channel_id = "" #@param {type:"string"} # Get your channel ID here https://commentpicker.com/youtube-channel-id.php

def get_channel_videos(channel_id, api_key):
    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key)

    video_ids = []
    page_token = None

    while True:
        request = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            maxResults=50,  # Fetch 50 videos at a time
            pageToken=page_token  # Add pagination
        )
        response = request.execute()

        video_ids += [item['id']['videoId'] for item in response['items'] if item['id']['kind'] == 'youtube#video']

        # Check if there are more videos to fetch
        if 'nextPageToken' in response:
            page_token = response['nextPageToken']
        else:
            break

    return video_ids

def get_transcripts(video_ids):
    transcripts = []
    for video_id in tqdm(video_ids):
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            transcripts.append(transcript)
        except Exception as ex:
            print(f"An error occurred for video: {video_id} [{ex}]")
    return transcripts

def write_to_file(transcripts):
    with open('YouTube.txt', 'w') as f:
        for transcript in transcripts:
            for item in transcript:
                f.write(item['text'] + '\n')

def main(api_key, channel_id):
    video_ids = get_channel_videos(channel_id, api_key)[:20]
    transcripts = get_transcripts(video_ids)
    print(transcripts)
    write_to_file(transcripts)

main(api_key, channel_id)

### Run our text file through infiniteGPT to clean its grammar and punctuation.

In [ ]:
YOUR_OPENAI_API_KEY = "" #@param {type:"string"}
import openai
from langchain.chat_models.openai import ChatOpenAI
from concurrent.futures import ThreadPoolExecutor
import tiktoken
from pathlib import Path
from langchain.schema import (
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(model="gpt-3.5-turbo",temperature=0.2,max_tokens=500)

openai.api_key = YOUR_OPENAI_API_KEY

def load_text(file_path):
    with Path(file_path).open("r") as file:
        return file.read()

def save_to_file(responses, output_file):
    with Path(output_file).open('w') as file:
        file.write("\n".join(responses))

def call_openai_api(chunk):
    messages = [
        SystemMessage(content="Clean the following transcripts of all gramatical mistakes, misplaced words, and identify the speakers."),
        HumanMessage(content=chunk)
    ]
    response = chat(messages)
    return response.content.strip()

def split_into_chunks(text, n_tokens=300):
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), n_tokens):
        chunks.append(' '.join(encoding.decode(tokens[i:i + n_tokens])))
    return chunks

def process_chunks(input_file, output_file, delay=0):  # delay in seconds (if you hit a rate limit error)
    text = load_text(input_file)
    chunks = split_into_chunks(text)[:5]
    responses = []
    for chunk in tqdm(chunks):
        responses.append(call_openai_api(chunk))

    save_to_file(responses, output_file)

    # Specify your input and output files

if __name__ == "__main__":
    input_file = "YouTube.txt"
    output_file = "clean_transcript.txt"
    process_chunks(input_file, output_file)

    # Can take up to a few minutes to run depending on the size of your data input

## Building the Knowledge Base

We start by constructing our knowledge base. Make sure to input the correct .txt file. For most users, this will be your cleaned_transcript.txt file.

In [ ]:
import pandas as pd

# Read the .txt file into Python
with Path('clean_transcript.txt').open('r') as file:
    lines = file.read().splitlines()

# Group the lines into chunks of 5
chunks = [' '.join(lines[i:i+5]) for i in range(0, len(lines), 5)]

# Convert list of chunks into a DataFrame
data = pd.DataFrame(chunks, columns=['context'])

# Add an index column and a name column
data['name'] = 'youtube'

# Remove duplicates (if any)
data.drop_duplicates(subset='context', keep='first', inplace=True)

print(data)

### Initialize the Embedding Model and Vector DB

We'll be using OpenAI's `text-embedding-ada-002` model initialize via LangChain and the Pinecone vector DB. We start by initializing the embedding model, for this we need an [OpenAI API key](https://platform.openai.com/).

*(Note that OpenAI is a paid service and so running the remainder of this notebook will cost a few dimes)*

In [ ]:
from getpass import getpass
from langchain.embeddings.openai import OpenAIEmbeddings

OPENAI_API_KEY = getpass("OpenAI API Key: ")  # platform.openai.com
model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

Next we initialize the vector database. For this we need a [free API key](https://app.pinecone.io/), then we create the index:

In [ ]:
 import pinecone

# find API key in console at app.pinecone.io

YOUR_API_KEY = '' #@param {type:"string"}
# find ENV (cloud region) next to API key in console
YOUR_ENV = '' #@param {type:"string"}

index_name = 'youtube-chatbot-agent'
pinecone.init(
    api_key=YOUR_API_KEY,
    environment=YOUR_ENV
)

if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(
        name=index_name,
        metric='dotproduct',
        dimension=1536  # 1536 dim of text-embedding-ada-002
    )

Then connect to the index:

In [ ]:
index = pinecone.GRPCIndex(index_name)
index.describe_index_stats()

We should see that the new Pinecone index has a `total_vector_count` of `0`, as we haven't added any vectors yet.

## Indexing

We can perform the indexing task using the Pinecone python client directly. We will do this in batches of `100` or more.

In [ ]:
from tqdm.auto import tqdm
from uuid import uuid4

# Reset index and ensure 'index' column is added
data = data.reset_index(drop=True)
data = data.reset_index()

batch_size = 100

for i in tqdm(range(0, len(data), batch_size)):
    # get end of batch
    i_end = min(len(data), i+batch_size)
    batch = data.iloc[i:i_end]

    # first get metadata fields for this record
    metadatas = [{
      'text': record[0],  # 'text' will contain the same data as 'context'
      'name': record[1]
    } for record in batch.itertuples(index=False)]

    # get the list of contexts / documents
    documents = batch['context'].tolist()

    # create document embeddings
    embeds = embed.embed_documents(documents)

    # get IDs and convert them to strings
    ids = batch['index'].astype(str).tolist()

    # add everything to pinecone
    index.upsert(vectors=list(zip(ids, embeds, metadatas)))


  0%|          | 0/9 [00:00<?, ?it/s]

We've indexed everything, now we can check the number of vectors in our index like so:

In [ ]:
index.describe_index_stats()

## Creating a Vector Store and Querying

Now that we've build our index we can switch back over to LangChain. We start by initializing a vector store using the same index we just built. We do that like so:

In [ ]:
from langchain.vectorstores import Pinecone

text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

As in previous examples, we can use the `similarity_search` method to do a pure semantic search (without the generation component).

In [ ]:
query = "how to make viral videos?"

vectorstore.similarity_search(
    query,  # our search query
    k=3  # return 3 most relevant docs
)

Looks like we're getting good results. Let's take a look at how we can begin integrating this into a conversational agent.

## Initializing the Conversational Agent

Our conversational agent needs a Chat LLM, conversational memory, and a `RetrievalQA` chain to initialize. We create these using:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA

# chat completion llm
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

Using these we can generate an answer using the `run` method:

In [ ]:
qa.run(query)

But this isn't yet ready for our conversational agent. For that we need to convert this retrieval chain into a tool. We do that like so:

In [ ]:
from langchain.agents import Tool

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool for every query to get '
            'more information and stories on the topic'
        )
    )
]

Now we can initialize the agent like so:

In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

With that our retrieval augmented conversational agent is ready and we can begin using it. For better results, make the sys_msg description as specific and elaborate as possible.

In [ ]:
agent.agent.llm_chain.prompt

In [ ]:
sys_msg = """You are [NAME], a [description] & content creator. You are an expert in [description]. Answer the user's questions with [desired tone].
"""

prompt = agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
agent.agent.llm_chain.prompt = prompt

In [ ]:
agent.agent.llm_chain.prompt

### Using the Conversational Agent

To make queries we simply call the `agent` directly.

In [ ]:
agent("Tell me about yourself")

We're also able to ask questions that refer to previous interactions in the conversation and the agent is able to refer to the conversation history to as a source of information.

Once finished, delete the Pinecone index to save resources:

In [ ]:
pinecone.delete_index(index_name)

---

In [ ]:
# Install dependencies Hòa test
!pip install -qU \
    pinecone-client[grpc]==2.2.1 \
    langchain==0.0.162 \
    tiktoken==0.4.0 \
    datasets==2.12.0 \
    youtube_transcript_api \
    google-api-python-client \
    google-generativeai

# Scrape an entire YouTube channel into one .txt file

import googleapiclient.discovery
from tqdm import tqdm
from youtube_transcript_api import YouTubeTranscriptApi
# Make sure to replace with your actual API key
api_key = ""  #@param {type:"string"}
# Replace with the actual channel ID
channel_id = ""  #@param {type:"string"}

def get_channel_videos(channel_id, api_key):
    youtube = googleapiclient.discovery.build(
        "youtube", "v3", developerKey=api_key)

    video_ids = []
    page_token = None

    while True:
        request = youtube.search().list(
            part="snippet",
            channelId=channel_id,
            maxResults=50,  # Fetch 50 videos at a time
            pageToken=page_token  # Add pagination
        )
        response = request.execute()

        video_ids += [item['id']['videoId'] for item in response['items'] if item['id']['kind'] == 'youtube#video']

        # Check if there are more videos to fetch
        if 'nextPageToken' in response:
            page_token = response['nextPageToken']
        else:
            break

    return video_ids

def get_transcripts(video_ids):
    transcripts = []
    for video_id in tqdm(video_ids):
        try:
            transcript = YouTubeTranscriptApi.get_transcript(video_id)
            transcripts.append(transcript)
        except Exception as ex:
            print(f"An error occurred for video: {video_id} [{ex}]")
    return transcripts

def write_to_file(transcripts):
    with open('YouTube.txt', 'w') as f:
        for transcript in transcripts:
            for item in transcript:
                f.write(item['text'] + '\n')

def main(api_key, channel_id):
    video_ids = get_channel_videos(channel_id, api_key)[:20]
    transcripts = get_transcripts(video_ids)
    print(transcripts)
    write_to_file(transcripts)

main(api_key, channel_id)

### Run our text file through infiniteGPT to clean its grammar and punctuation.
import os
import google.generativeai as genai
import tiktoken
from pathlib import Path
from tqdm import tqdm

# Set your Gemini API key.  You can also set it as an environment variable.
# Make sure to replace with your actual API key
GOOGLE_API_KEY = "-McF0sSsk"  #@param {type:"string"}
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

model = genai.GenerativeModel('gemini-1.5-pro-latest')  # Or another suitable Gemini model

def load_text(file_path):
    with Path(file_path).open("r") as file:
        return file.read()

def save_to_file(responses, output_file):
    with Path(output_file).open('w') as file:
        file.write("\n".join(responses))

def call_gemini_api(chunk):
    """Calls the Gemini API to clean the given chunk of text."""
    prompt = f"""Clean the following transcripts of all grammatical mistakes, misplaced words, and identify the speakers:
    {chunk}
    """

    try:
        response = model.generate_content(prompt)
        response.resolve() # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

def split_into_chunks(text, n_tokens=300):
    encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')  # Using gpt-3.5-turbo tokenizer for token estimation
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), n_tokens):
        chunk_tokens = tokens[i:i + n_tokens]
        try:
            chunks.append(encoding.decode(chunk_tokens))
        except Exception as e:
            print(f"Error decoding tokens: {e}")
            # Handle the error, perhaps by skipping this chunk or using a different decoding strategy
            continue
    return chunks

def process_chunks(input_file, output_file, delay=0):  # delay in seconds (if you hit a rate limit error)
    text = load_text(input_file)
    chunks = split_into_chunks(text)[:5]
    responses = []
    for chunk in tqdm(chunks):
        responses.append(call_gemini_api(chunk))

    save_to_file(responses, output_file)

    # Specify your input and output files

if __name__ == "__main__":
    input_file = "YouTube.txt"
    output_file = "clean_transcript.txt"
    process_chunks(input_file, output_file)

## Building the Knowledge Base

# Read the .txt file into Python
with Path('clean_transcript.txt').open('r') as file:
    lines = file.read().splitlines()

# Group the lines into chunks of 5
chunks = [' '.join(lines[i:i+5]) for i in range(0, len(lines), 5)]

### Initialize the Embedding Model and Vector DB

#No longer use OpenAI Embeddings. We pass transcripts directly to the model.

### Initialize the Conversational Agent

# conversational memory. No longer using vector store
conversational_memory = []

def clean_query(query):
    prompt = f"""Clean the following query to make sure it is gramatically correct and can retrieve the correct information: {query}"""
    try:
        response = model.generate_content(prompt)
        response.resolve() # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully


def query_knowledge_base(query, knowledge_base):
    """
    Queries the knowledge base using the Gemini API.
    """
    context = "\n".join(knowledge_base)
    prompt = f"""You are a chatbot that answers question to the best of your ability using the context provided
    Context:
    {context}

    Question: {query}
    """
    try:
        response = model.generate_content(prompt)
        response.resolve() # Ensure the response is ready
        return response.text.strip()
    except Exception as e:
        print(f"Error during Gemini API call: {e}")
        return ""  # Handle errors gracefully

def conversational_agent(query, knowledge_base, chat_history):
    """
    A conversational agent that uses the Gemini API to answer questions.
    """

    # Clean query before processing
    cleaned_query = clean_query(query)

    # Query the knowledge base
    response = query_knowledge_base(cleaned_query, knowledge_base)

    # Update the chat history
    chat_history.append(f"User: {query}")
    chat_history.append(f"Assistant: {response}")

    return response, chat_history

### Using the Conversational Agent

# Example usage
knowledge_base = chunks.copy() #copy chunks text snippets to knowledge base
chat_history = [] #Initialize empty chat history

query1 = "Tell me about yourself"
response1, chat_history = conversational_agent(query1, knowledge_base, chat_history)
print(response1)

query2 = "What are some strategies for growing an audience?"
response2, chat_history = conversational_agent(query2, knowledge_base, chat_history)
print(response2)

query3 = "Can you summarize what we've talked about?"
response3, chat_history = conversational_agent(query3, knowledge_base, chat_history)
print(response3)

print(chat_history)

# Once finished, delete the Pinecone index to save resources:

# pinecone.delete_index(index_name) #No longer using vector database